# Inventory

## Content

<span style='background:yellow'> 2.1) Setting up the background system: ecoinvent and biosphere3 databases </span> 

<span style='background:yellow'> 2.2) Setting up the foreground system: </span> 




## 2.1) Setting up the background system: ecoinvent and biosphere3 databases

You will need two databases: 
- biosphere3, that contains all the exchanges and impact assessment methods, and
- ecoinvent.

But first, let's import BW25 packages

In [3]:
# Import BW25 packages
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

/Users/andreavargasf/anaconda3/envs/STI_environment/lib/python3.11/site-packages/bw2calc/__init__.py:48: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


### Biosphere3

#### Importing biosphere3

Check the projects you have created:

In [4]:
bd.projects 

Brightway2 projects manager with 7 objects:
	2024_STI_BW25_Lecture
	asphalt
	beavers
	default
	example
	spm
	ua_pmlca
Use `projects.report()` to get a report on all projects.

In [5]:
# Activate the project we are working on
bd.projects.set_current('2024_STI_BW25_Lecture') 

 Let’s see what databases you have:

In [6]:
# If this is your first time using this project, this should be an empty dictionary! 
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

Now we can add the biosphere flows. This is done by running the bw2setup function:

(This will also create the LCIA methods. But more about that will be explored later)

In [5]:
bi.bw2setup()

Biosphere database already present!!! No setup is needed


Check the databases. You should now see "biosphere3"

In [6]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

#### Navigating biosphere3

There are many properties and functions associated with this database object. To access those, let's assign the database to a variable

In [7]:
my_bio = bd.Database('biosphere3')

In [8]:
#type(my_bio)
#len(my_bio)

If you type my_bio. and click on tab, you should get a list of properties and methods associated with database objects. 

In [9]:
my_bio.get

<bound method SQLiteBackend.get of Brightway2 SQLiteBackend: biosphere3>

Some of the more basic ones we will be using now are :

- random() - returns a random activity in the database
- get(*valid_exchange_tuple*) - returns an activity, but you must know the activity key
- load() - loads the whole database as a dictionary.
- make_searchable - allows searching of the database (by default, it is already searchable)
- search - search the database

Let's start with random:

In [10]:
my_bio.random()

'Ethanol' (kilogram, None, ('air',))

This returns a biosphere activity, but without assigning it to a variable, there is not much we can do with it directly.

Note: It may seem counter-intuitive for elementary flows to be considered activities in Brightway, but it is no mistake. LCA models are made up of nodes (activities) that are linked by edges (exchanges). The biosphere activities are the nodes outside the technosphere. Emissions and resource extractions are modelled as exchanges between activities in the technosphere (part of the product system) and these biosphere activities.

More on this later.

For now, let's assign another random activity to a variable:

In [11]:
random_biosphere = my_bio.random()
random_biosphere

'Haloxyfop-ethoxyethyl' (kilogram, None, ('water', 'ground-'))

We can get the type of the object that was returned from the database:

In [12]:
type(random_biosphere)

bw2data.backends.proxies.Activity

The type is an activity proxy. Activity proxies allow us to interact with the content of the database.

To see what the activity contains, we can convert it to a dictionary:

In [13]:
random_biosphere.as_dict()

{'categories': ('water', 'ground-'),
 'code': '125cffa2-ba58-5995-8645-24e7789cdccc',
 'CAS number': '087237-48-7',
 'synonyms': [],
 'name': 'Haloxyfop-ethoxyethyl',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission',
 'id': 4410}

We can get the value of each item by specifying its key. Example: let's get the value corresponding to the key 'code'

In [14]:
random_biosphere['code']

'125cffa2-ba58-5995-8645-24e7789cdccc'

Now for the key 'unit'

In [15]:
random_biosphere['unit']

'kilogram'

Activities in the biosphere3 database have unique codes, which we can use with the get function:

In [16]:
my_bio.get(random_biosphere['code'])

'Haloxyfop-ethoxyethyl' (kilogram, None, ('water', 'ground-'))

As you can see this is the same activity! This is expected because we are "getting" the activity with the code equal to our random biosphere variable. And remember: Activities in the biosphere3 database have unique codes.

You can always find (or return) the key to an activity using the .key property.

In [17]:
random_biosphere.key

('biosphere3', '125cffa2-ba58-5995-8645-24e7789cdccc')

**Searching for activities**

Let's say we are looking for a specific elementary flow. For this purpose we can use the search method of the database

In [18]:
bd.Database('biosphere3').search('carbon dioxide')

['Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil, resource correction' (kilogram, None, ('natural reso

We can do the same operation by using the variable we assigned previously to the biosphere database

In [19]:
my_bio.search('carbon dioxide')

['Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil, resource correction' (kilogram, None, ('natural reso

It is also possible to use "filters" to narrow searches, e.g.

In [20]:
my_bio.search('carbon dioxide', filter={'categories':'urban', 'name':'fossil'})

Excluding 17 filtered results


['Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))]

We can also use "home-made" searches with list comprehensions. This approach allow us to add as many criteria as we wante

In [21]:
[act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ]

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))]

Activities returned by searches or list comprehensions can be assigned to variables, but to do so, one needs to identify the activity by index. Based on the above, we can refine the filters to ensure the list comprehension only returns one activity, and that that activity is the one we really want.

In [22]:
activity_I_want = [act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ][0]

In [23]:
print(activity_I_want)

'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))


####

**LCIA Methods**

As mentioned before bi.bw2setup() also installed LCIA methods. LCIA methods are a set of principles, models and characteriation factors that convert biosphere flows into environment impacts scores.

Run the code below to know the methods we have access to:

In [24]:
list(bd.methods)
#bd.methods.list #this produces the same outputs

[('CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'energy resources: non-renewable no LT',
  'abiotic depletion potential (ADP): fossil fuels no LT'),
 ('CML v4.8 2016 no LT',
  'eutrophication no LT',
  'eutrophication (fate not incl.) no LT'),
 ('CML v4.8 2016 no LT',
  'human toxicity no LT',
  'human toxicity (HTP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'material resources: metals/minerals no LT',
  'abiotic depletion potential (ADP): elements (ultimate reserves) no LT')

For each method, the 1st name is the name of the impact assessment methodology, the 2nd name is the name of the impact category, and the 3rd name is the name of the characterization model. Let's take as an exmaple ('CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)': 

- 'CML v4.8 2016' = impact assessment methodology
- 'acidification' = impact category
- 'acidification (incl. fate, average Europe total, A&B)' = characterization model/method


We get more details about the LCIA methods. Those details are stored as values (bw.methods.values()), with the LCIA method names being the keys (bw.methods.keys()):

In [25]:
for key in bd.methods:
    print(key, ':', bd.methods[key])

('CML v4.8 2016 no LT', 'acidification no LT', 'acidification (incl. fate, average Europe total, A&B) no LT') : {'description': '', 'filename': 'LCIA_Implementation_3.9.xlsx', 'unit': 'kg SO2-Eq', 'abbreviation': 'cml-v48-2016-no-ltaa.9326888b5dfc8ad763461e87dc5a0a9e', 'num_cfs': 21, 'geocollections': ['world']}
('CML v4.8 2016 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT') : {'description': '', 'filename': 'LCIA_Implementation_3.9.xlsx', 'unit': 'kg CO2-Eq', 'abbreviation': 'cml-v48-2016-no-ltcg.ef26e472ac5329eeba834f793ba181a9', 'num_cfs': 149, 'geocollections': ['world']}
('CML v4.8 2016 no LT', 'ecotoxicity: freshwater no LT', 'freshwater aquatic ecotoxicity (FAETP inf) no LT') : {'description': '', 'filename': 'LCIA_Implementation_3.9.xlsx', 'unit': 'kg 1,4-DCB-Eq', 'abbreviation': 'cml-v48-2016-no-ltef.7bcf2b54b0276df08edfbd557353fcec', 'num_cfs': 692, 'geocollections': ['world']}
('CML v4.8 2016 no LT', 'ecotoxicity: marine no LT', 'marine aquatic eco

To access the description of only one characterization method, type:

In [26]:
bd.methods.get(('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'))

{'description': '',
 'filename': 'LCIA_Implementation_3.9.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'cml-v48-2016cg.231d6e8f8b1c199a47182515eba4032e',
 'num_cfs': 185,
 'geocollections': ['world']}

We can also get the value of an item by specifying the corresponding key. Let's imagine we want to know the units considered by the characterization method:

In [27]:
bd.methods.get(('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')).get('unit')

'kg CO2-Eq'

As we did with the biosphere activities, we can also assign variables to characterization methods:

In [28]:
my_method = bd.methods.get(('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'))
type(my_method)

dict

We can then get the value corresponding to the key we are interested in. Let's consider again we want to know the unitsof the characterization method:

In [29]:
my_method['unit']

'kg CO2-Eq'

We can get the same information by running the code below:

In [30]:
my_method.get('unit')

'kg CO2-Eq'

We can also search for a specific method using list comprehensions. In your group assignment you will work with a LCIA methodology called CML, and specifically the version 4.8, released in 2016. To know all impact categories and respective characterization methods considered by CML v4.8, we can type:

In [31]:
[m for m in bd.methods if 'CML v4.8 2016' in str(m)]

[('CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'energy resources: non-renewable no LT',
  'abiotic depletion potential (ADP): fossil fuels no LT'),
 ('CML v4.8 2016 no LT',
  'eutrophication no LT',
  'eutrophication (fate not incl.) no LT'),
 ('CML v4.8 2016 no LT',
  'human toxicity no LT',
  'human toxicity (HTP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'material resources: metals/minerals no LT',
  'abiotic depletion potential (ADP): elements (ultimate reserves) no LT')

The query above resulted in two "types" of CML. One is what we were looking for ('CML v4.8 2016') and another one is 'CML v4.8 2016 no LT', where LT stands for Land Transformation. We can get exactly the one we are looking for by refining the search as follows:

In [32]:
[m for m in bd.methods if 'CML v4.8 2016' in str(m) and not 'no LT' in str(m)]

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

In [33]:
Ozone_depletion = [m for m in bd.methods if 'ReCiPe 2016 v1.03, midpoint (E) no LT' 
                   in str(m) and 'ozone depletion no LT' in str(m)][0]
type(Ozone_depletion)

tuple

We can also set the characterization method as an object and benefit from the multiple 'Methods' associated with a pyhton object. In pyhton, methods are functions associated with an object, and can manipulate and perform action on it. et consider in are interested in the impact category Climate Change and respective characterization method, considered by 'CML v4.8 2016':

In [34]:
cml_ClimateChange_method = bd.Method(('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'))

In [35]:
type(cml_ClimateChange_method)

bw2data.method.Method

In [36]:
cml_ClimateChange_method.metadata

{'description': '',
 'filename': 'LCIA_Implementation_3.9.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'cml-v48-2016cg.231d6e8f8b1c199a47182515eba4032e',
 'num_cfs': 185,
 'geocollections': ['world']}

In [37]:
cml_ClimateChange_method.metadata['unit']

'kg CO2-Eq'

In [38]:
cml_ClimateChange_method.metadata.get('unit')

'kg CO2-Eq'

We can get the elementary flows and corresponding CFs associated witht the LCIA method by doing the following:

In [39]:
cml_ClimateChange_method.load()

[(['biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'], 1),
 (['biosphere3', '4cadbc9f-0472-4bb9-a942-df5b90e37878'], 16),
 (['biosphere3', 'fa0c2bee-8dd9-4f8a-8489-b1f3b43de958'], 1300),
 (['biosphere3', '818cee9e-231c-4b53-8ed2-47a0001802d5'], 160),
 (['biosphere3', 'f43b2b80-f5a6-4d31-9980-0754305a978d'], 4800),
 (['biosphere3', '033d3a16-e1a2-4ce8-8769-f4ea867801ca'], 782),
 (['biosphere3', '393d0862-04cc-45ae-8d08-9dc2a4461b35'], 138),
 (['biosphere3', '400be66e-4383-482a-93e3-c005e96de261'], 1980),
 (['biosphere3', 'db907ff7-6cf0-4f88-9376-116262dfad6d'], 79),
 (['biosphere3', '27f61651-96a5-45c7-9873-832626cf6905'], 527),
 (['biosphere3', '15e54d5b-aba2-40a6-9aa8-aa2b8c432129'], 7670),
 (['biosphere3', 'df5dd437-2e12-4af6-8f7a-9c8224857dc5'], 11100),
 (['biosphere3', '5d1701c5-c4ef-49c7-8a68-ce7c255eee9f'], 3170),
 (['biosphere3', '4bf1c333-419b-4278-ac14-247c8bf563d6'], 2),
 (['biosphere3', '816c43d1-bc30-4a72-81db-08cb6be4da4f'], 1750),
 (['biosphere3', '18ddd879-5496-4419-8

Lets create a dictionary that has as key the name of the elemenyary flows and as value the corresponding CF:

<span style='background:red'> (!!!!! code not working) </span> 

In [40]:
OD_dict = {bd.get_activity(ef[0])['name']:ef[1] for ef in cml_ClimateChange_method.load()}

MultipleResults: Found 25947 results for the given search

In [41]:
OD_dict = {my_bio.get(ef[0])['name']:ef[1] for ef in cml_ClimateChange_method.load()}

OperationalError: row value misused

### Ecoinvent

#### Importing Ecoinvent

You need to get the ecoinvent files first. To do so, follow the following steps:
- Go to STI course's Canvas page -> Module <span style='background:yellow'> "XXXXXX" </span>
- Download the file ecoinvent <span style='background:yellow'> XXXXXX </span>  in a folder of yours. Make sure you remember the full path to this directory.
- The file you have downloaded is a compressed archive of many files (like with winzip or winrar). Extract the files from the .7z archive, e.g. by double clicking it. If it does not work, install a software that can do that. 
Now you an run the cells below. Make sure you change the path line and replace it with the one where you have extracted the files. For example, I have extracted the files in a folder called "datasets". The path to this folder is: */Users/joaosantos/Library/CloudStorage/OneDrive-UniversityofTwente/UT_Academic Life/My Software Purchase/ecoinvent v3.9.1/datasets*  You will see this same line in the script and you need to change it with your directory.

<span style='background:yellow'> https://medium.com/game-of-data/12-things-to-know-about-jupyter-notebook-markdown-3f6cef811707 </span>


Let’s see the projects you have created:

In [42]:
bd.projects #Checked the projects you created

Brightway2 projects manager with 5 objects:
	2024_STI_BW25_Lecture
	Test_location_specifc_folder
	bw25_20240406
	bw25_test
	default
Use `projects.report()` to get a report on all projects.

In [43]:
# Activate the project we are working on
bd.projects.set_current('2024_STI_BW25_Lecture') 

 Let’s see what databases we have:

In [44]:
# If this is your first time using this project, this should be an empty dictionary! 
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

Let's import the ecoinvent database

In [45]:
#directory where the ecoinvent is saved
ei_path = "ecoinvent 3.9.1_cutoff_ecoSpold02/datasets"

# Give the database a name 
# In this course we will use ecoinvent v3.9.1 cutoff.
ei_name = "ecoinvent-391-cutoff"

if ei_name in bd.databases:
    print("Database has already been imported")
else:
    # Go ahead and import:
    ei_importer = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
    # Apply stragegies 
    ei_importer.apply_strategies()
    # We can get some statistics
    ei_importer.statistics()
    # Now we will write the database into our project. 
    ei_importer.write_database()
    

Database has already been imported


Let's see the databases we have in our working project:

In [46]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

The db we have just imported in now in our project. Let's assign it to a variable:

In [47]:
eidb = bd.Database(ei_name)

In [48]:
print("The imported ecoinvent database is of type {} and has a length of {}.".format(type(eidb), len(eidb)))

The imported ecoinvent database is of type <class 'bw2data.backends.base.SQLiteBackend'> and has a length of 21238.


#### Navigating ecoinvent

##### LCI activities

In the context of LCI databases, activities are the nodes "within the technosphere". They are therefore the columns in the technosphere matrix A.

There are different ways to get access to an activity. Similarly to the biosphere database, we can use the random() method  to explore a random activity in the ecoinvent database.

In [49]:
random_act = eidb.random()
print(random_act)

'heat and power co-generation, wood chips, 6667 kW' (kilowatt hour, BR-Southern grid, None)


In [50]:
type(random_act)

bw2data.backends.proxies.Activity

To know what is stored in an activity object, let's convert our random act in a dictionary:

In [51]:
from pprint import pprint
pprint(random_act.as_dict())

{'activity': 'aae65b44-fcc0-5f60-bf04-7ba1f93ef06d',
 'activity type': 'ordinary transforming activity',
 'authors': {'data entry': {'email': 'no@email.com', 'name': '[Current User]'},
             'data generator': {'email': 'no@email.com',
                                'name': '[Current User]'}},
 'classifications': [('ISIC rev.4 ecoinvent',
                      '3530:Steam and air conditioning supply'),
                     ('CPC', '17100: Electrical energy')],
 'code': '7d6c3f5016a7ee860718eb7adede39a0',
 'comment': 'This dataset represents the production of heat and electricity '
            'with wood chips in a co-generation plant with a capacity of 6667 '
            'kW (referring to fuel input) in Switzerland and was used as '
            'approximation for this dataset valid for in Brazil Southern grid '
            '(uncertainties adjusted accordingly). The technology corresponds '
            'to the average installed in 2014 in Switzerland, which is '
            'equi

Another way to get the same info consists of using the method _data. 

In [52]:
random_act._data

{'comment': 'This dataset represents the production of heat and electricity with wood chips in a co-generation plant with a capacity of 6667 kW (referring to fuel input) in Switzerland and was used as approximation for this dataset valid for in Brazil Southern grid (uncertainties adjusted accordingly). The technology corresponds to the average installed in 2014 in Switzerland, which is equipped with an electrostatic precipitator for particulate matter (PM) emission reduction. It is assumed that the plant is operated mainly producing heat, i.e. following the heat demand. Heat is therefore considered as the main product and the reference product, whereas electricity is a by-product. Electricity is produced with an organic rankine cycle (ORC) steam generator. \nWood chips are burned in a boiler at a temperature of 800-1300 °C under excess air conditions and turned into carbon dioxide and water. Energy is set free in form of heat. Burning takes place with the following steps:\n1. Drying\n2

We can get acess to the value of each key by using the method .get(). Let's take as an example the key 'reference product'. The corresponding value is retrieved by running the code below:

In [53]:
random_act.get('reference product')

'electricity, high voltage'

Two alternative ways to obtain the same info are given below:

In [56]:
random_act._data['reference product']

'electricity, high voltage'

In [57]:
random_act['reference product']

'electricity, high voltage'

We can also use the search method to search for specific activities. Let's search for three activities that commonly used in pavement LCA studies, namely the (1) production of limestone, (2) production of bitumen, and (2) transportation 

In [58]:
# Using search
eidb.search('limestone')

['limestone quarry operation' (kilogram, IN, None),
 'limestone quarry operation' (kilogram, CA-QC, None),
 'market for limestone, unprocessed' (kilogram, CH, None),
 'limestone quarry operation' (kilogram, RoW, None),
 'market for limestone, unprocessed' (kilogram, IN, None),
 'limestone quarry operation' (kilogram, CH, None),
 'market for limestone residue' (kilogram, CH, None),
 'limestone quarry construction' (unit, CH, None),
 'recultivation, limestone mine' (square meter, CH, None),
 'recultivation, limestone mine' (square meter, RoW, None),
 'market for limestone, unprocessed' (kilogram, RoW, None),
 'market for limestone residue' (kilogram, RoW, None),
 'limestone production, crushed, washed' (kilogram, CA-QC, None),
 'limestone quarry construction' (unit, RoW, None),
 'limestone production, crushed, washed' (kilogram, CH, None),
 'market for limestone, crushed, washed' (kilogram, CH, None),
 'market for limestone, crushed, washed' (kilogram, RoW, None),
 'limestone production,

The code above retrieves multiple activities involving limestone. We can refine the search by using filter with as many criteria as we want (provided that they are valid):

In [59]:
# Using search
act_name = 'limestone' 
eidb.search(act_name, filter={'location':'ROW', 'name':'crushed'})


Excluding 257 filtered results


['market for limestone, crushed, washed' (kilogram, RoW, None),
 'limestone production, crushed, washed' (kilogram, RoW, None),
 'market for limestone, crushed, for mill' (kilogram, RoW, None),
 'limestone production, crushed, for mill' (kilogram, RoW, None),
 'limestone, crushed, washed to generic market for supplementary cementitious materials' (kilogram, RoW, None),
 'gravel production, crushed' (kilogram, RoW, None)]

...refining the search a bit more:

In [60]:
eidb.search(act_name, filter={'location':'ROW', 'name':'crushed', 'name':'washed'})

Excluding 260 filtered results


['market for limestone, crushed, washed' (kilogram, RoW, None),
 'limestone production, crushed, washed' (kilogram, RoW, None),
 'limestone, crushed, washed to generic market for supplementary cementitious materials' (kilogram, RoW, None)]

It might be useful to know the code of the activities retrived. That can be accomplished by running the following code:

In [61]:
for activity in eidb.search(act_name, filter={'location':'RoW', 'name':'washed','name':'crushed'}):
    print(activity)
    print(activity['code'])

Excluding 257 filtered results
'market for limestone, crushed, washed' (kilogram, RoW, None)
f02db7dc2aa90b15c39e38e5b2871a63
'limestone production, crushed, washed' (kilogram, RoW, None)
570143a9c955c81d5f8bc1e9bd8d22a1
'market for limestone, crushed, for mill' (kilogram, RoW, None)
80668cba452d7daa47ec2b932c503fd2
'limestone production, crushed, for mill' (kilogram, RoW, None)
c0505f1c9c2dc2f1a86b087089a33987
'limestone, crushed, washed to generic market for supplementary cementitious materials' (kilogram, RoW, None)
16f5ec192027775224682821f0453d17
'gravel production, crushed' (kilogram, RoW, None)
0cff374b6cd6c946c75f0a8dc694ab66


This search resulted in three activities. Let's image we are interested in selecting the activity 'limestone production, crushed, washed', which is the second of the list, and we want to assign it to a variable called my_act. Invariably, there are many ways to do this. One of them is presented below:

In [62]:
my_act = eidb.get('f02db7dc2aa90b15c39e38e5b2871a63')
print(my_act)


'market for limestone, crushed, washed' (kilogram, RoW, None)


The search method can also be used with list comprehension:

In [63]:
[act for act in eidb if 'limestone' in act['name']
        and 'crushed' in act['name']
        and 'washed' in act['name']
        and 'RoW' in act['location']
]

['market for limestone, crushed, washed' (kilogram, RoW, None),
 'limestone production, crushed, washed' (kilogram, RoW, None),
 'limestone, crushed, washed to generic market for supplementary cementitious materials' (kilogram, RoW, None)]

In [64]:
[act for act in eidb if 'limestone' in act['name']
        and 'crushed' in act['name']
        and 'washed' in act['name']
        and 'RoW' in act['location']
][0]

'limestone production, crushed, washed' (kilogram, RoW, None)

**Exercise**: Select the activities that best represent: 
- The production of bitumen used in asphalt mixtures;
- The transportation of materials used in road paving activities

##### LCI exchanges

Exchanges are the edges between nodes.

These can be:

- an edge between two activities within the technosphere (an element aij of A matrix)
- edges between an activity in the technosphere and an activity in the "biosphere" (an element bkj of the biosphere matrix B).
  
One can iterate through all exchanges that have a given activity as output. Let's use as example the my_act selected above:

In [65]:
for exc in my_act.exchanges():
    print(exc)


Exchange: 1.0 kilogram 'market for limestone, crushed, washed' (kilogram, RoW, None) to 'market for limestone, crushed, washed' (kilogram, RoW, None)>
Exchange: 1.0 kilogram 'limestone production, crushed, washed' (kilogram, RoW, None) to 'market for limestone, crushed, washed' (kilogram, RoW, None)>
Exchange: 0.017352839318965644 ton kilometer 'market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None) to 'market for limestone, crushed, washed' (kilogram, RoW, None)>
Exchange: 0.0003583442198355759 ton kilometer 'market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None) to 'market for limestone, crushed, washed' (kilogram, RoW, None)>
Exchange: 0.0022888164611987796 ton kilometer 'market for transport, freight, lorry, unspecified' (ton kilometer, BR, None) to 'market for limestone, crushed, washed' (kilogram, RoW, None)>


We can also iterate through subsets of the exchanges:

- Technosphere exchanges: exchanges linking to other activities in the technosphere, activity.technosphere()
- Biosphere exchanges: i.e. elementary flows, linking to activities in the biosphere database activity.biosphere()
- Production exchange: the reference flow of the activity activity.production

Let's consider we want to learn more info about the second exchange of those listed out above. We can do that by assigning it to a variable:

In [66]:
my_act_exchange = [exc for exc in my_act.technosphere()][1]
my_act_exchange._data

{'flow': 'cfbce515-3f54-4411-ad9d-3d26b7faa15a',
 'type': 'technosphere',
 'name': 'transport, freight, lorry, unspecified',
 'classifications': {'CPC': ['6511: Road transport services of freight']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'}},
 'activity': '5985dd21-41a0-54ff-9099-307681a5cbfb',
 'unit': 'ton kilometer',
 'comment': 'transportation over 20 km, based on expert judgement.',
 'amount': 0.017352839318965644,
 'pedigree': {'reliability': 3,
  'completeness': 1,
  'temporal correlation': 5,
  'geographical correlation': 5,
  'further technological correlation': 4},
 'uncertainty type': 2,
 'loc': -4.053999136470282,
 'scale': 0.4516635916254486,
 'scale without pedigree': 0.34641016151377546,
 'input': ('ecoinvent-391-cutoff', '187ad56dca8970918d4b9bbbb8ffb41d'),
 'output': ('ecoinvent-391-cutoff', 'f02db7dc2aa90b15c39e38e5b2871a63')}

An alternative way to get the same info:

In [67]:
my_act_exchange.as_dict()

{'flow': 'cfbce515-3f54-4411-ad9d-3d26b7faa15a',
 'type': 'technosphere',
 'name': 'transport, freight, lorry, unspecified',
 'classifications': {'CPC': ['6511: Road transport services of freight']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'}},
 'activity': '5985dd21-41a0-54ff-9099-307681a5cbfb',
 'unit': 'ton kilometer',
 'comment': 'transportation over 20 km, based on expert judgement.',
 'amount': 0.017352839318965644,
 'pedigree': {'reliability': 3,
  'completeness': 1,
  'temporal correlation': 5,
  'geographical correlation': 5,
  'further technological correlation': 4},
 'uncertainty type': 2,
 'loc': -4.053999136470282,
 'scale': 0.4516635916254486,
 'scale without pedigree': 0.34641016151377546,
 'input': ('ecoinvent-391-cutoff', '187ad56dca8970918d4b9bbbb8ffb41d'),
 'output': ('ecoinvent-391-cutoff', 'f02db7dc2aa90b15c39e38e5b2871a63')}

In [68]:
my_act_exchange['type']

'technosphere'

In [69]:
# Amount, or weight of the edge
my_act_exchange.amount # The following code produces the same result: my_act_exchange['amount']

0.017352839318965644

In [70]:
# Activity the exchange terminates in
my_act_exchange.output

'market for limestone, crushed, washed' (kilogram, RoW, None)

In [71]:
# Activity the exchange stems from
my_act_exchange.input

'market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None)

In [72]:
[exc for exc in my_act.production()]

[Exchange: 1.0 kilogram 'market for limestone, crushed, washed' (kilogram, RoW, None) to 'market for limestone, crushed, washed' (kilogram, RoW, None)>]

This is the reference flow of the selected activity!

Let's assign a biosphere flow to a variable, and check the following:

- Is the output the same as for the technosphere exchange?
- From what database does the biosphere exchange come from?
- What is the amount of the exchange (i.e. the weight of the edge connecting the two activities)?
  
NOTE: If we get a  list index out of range error when trying to subscript our list comprehension, it means our list comprehension is empty, i.e. that there are no biosphere flows associated with the activity.

In [73]:
# Assign the exchange to a variable:
bio_exchange = [exc for exc in my_act.biosphere()][0]
bio_exchange

IndexError: list index out of range

In [74]:
bio_exchange.output

NameError: name 'bio_exchange' is not defined